In [1]:
import numpy as np
import cv2

In [2]:
corner_track_params=dict(maxCorners=10,qualityLevel=0.3,minDistance=7,blockSize=7)

In [3]:
lk_params=dict(winSize=(200,200), maxLevel=2, criteria= (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS,10,0.03))

In [6]:
cap=cv2.VideoCapture('../DATA/hand_move.mp4')


ret,prevFrame=cap.read()

prevGray=cv2.cvtColor(prevFrame,cv2.COLOR_BGR2GRAY)
prevPoints=cv2.goodFeaturesToTrack(prevGray,**corner_track_params)   #corners
type(prevPoints)
mask=np.zeros_like(prevFrame)   #for visualizing or drawing lines

##########################################
writer=cv2.VideoWriter();
width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer=cv2.VideoWriter("newVideo.mp4",cv2.VideoWriter_fourcc(*'DIVX'),15,(width,height)) 
##########################################################################################
while cap.isOpened():
    ret,frame=cap.read()
    if ret:
        
        frameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

        nextPoints, status, err = cv2.calcOpticalFlowPyrLK(prevGray,frameGray,prevPoints,None, **lk_params)


        good_new=nextPoints[status == 1]
        good_prev=prevPoints[status == 1]

        for i,(new,prev) in enumerate(zip(good_new,good_prev)):

            x_new,y_new= new.ravel()
            x_prev,y_prev=prev.ravel()

            mask=cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)

            frame=cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)


        img=cv2.add(frame,mask)
        
        cv2.imshow("tracking",img)
        writer.write(img)
        
    k=cv2.waitKey(1)
    if k == ord('q'):
        break
        
    prevGray=frameGray.copy()
    prevPoints=nextPoints.reshape(-1,1,2)
    

cv2.destroyAllWindows()
cap.release()
writer.release()